In [1]:
import rtdl
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import pandas as pd
test_path = '../../data_preprocessed/dev_in.csv'
df_test = pd.read_csv(test_path)

In [3]:
import numpy as np
X_test_np = np.asarray(df_test.iloc[:,6:])
X_test = torch.FloatTensor(X_test_np)

In [4]:
# Preprocess into tensors

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

lab_to_ind = {0.0: 0, 10.0: 1, 11.0: 2, 12.0: 3, 13.0: 4, 20.0: 5, 21.0: 6, 22.0: 7, 23.0: 8}
batch_size = 256

# Test
y_test = df_test['fact_cwsm_class']
y_test = torch.LongTensor(np.asarray([lab_to_ind[lab] for lab in y_test]))

test_ds = TensorDataset(X_test, y_test)
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

In [5]:
# Get the device

def get_default_device():
#     # Force cpu for now
#     return torch.device('cpu')
    if torch.cuda.is_available():
        print("Got CUDA!")
        return torch.device('cuda')
    else:
        print("No CUDA found")
        return torch.device('cpu')

device = get_default_device()

Got CUDA!


In [8]:
# Create the Feature Transformer Model
seed = 1
model = rtdl.ResNet.make_baseline(
    d_in=X_test.shape[1],
    d_main=512,
    d_hidden=1024,
    dropout_first=0.1,
    dropout_second=0.0,
    n_blocks=4,
    d_out=len(lab_to_ind)
)

model_path = f'./trained_models/ResNet/model{seed}.th'
model.load_state_dict(torch.load(model_path))
model.eval().to(device)

ResNet(
  (first_layer): Linear(in_features=123, out_features=512, bias=True)
  (blocks): Sequential(
    (0): Block(
      (normalization): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (linear_first): Linear(in_features=512, out_features=1024, bias=True)
      (activation): ReLU()
      (dropout_first): Dropout(p=0.1, inplace=False)
      (linear_second): Linear(in_features=1024, out_features=512, bias=True)
      (dropout_second): Dropout(p=0.0, inplace=False)
    )
    (1): Block(
      (normalization): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (linear_first): Linear(in_features=512, out_features=1024, bias=True)
      (activation): ReLU()
      (dropout_first): Dropout(p=0.1, inplace=False)
      (linear_second): Linear(in_features=1024, out_features=512, bias=True)
      (dropout_second): Dropout(p=0.0, inplace=False)
    )
    (2): Block(
      (normalization): BatchNorm1d(512, eps=1e-05, momen

In [9]:
# Create pipeline to apply model
def apply_model(model, x_num, x_cat=None):
    '''
    FTTransformer expects numerical and categorical inputs separately
    '''
    return model(x_num, x_cat) if isinstance(model, rtdl.FTTransformer) else model(x_num)

In [10]:
# Get model size
def count_parameters(model): return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(count_parameters(model))

4273673


In [11]:
@torch.no_grad()
def eval(val_loader, model, device):
    '''
    Run evaluation
    '''
    # switch to eval mode
    model.eval()
    preds = []
    
    for i, (x, target) in enumerate(val_loader):

        x = x.to(device)
        target = target.to(device)

        # Forward pass
        logits = apply_model(model, x)
        logits = logits.detach().cpu().numpy().tolist()
        preds += logits
    
    return preds

In [12]:
# Performance metrics

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_curve

def metric_accuracy(preds, targets):
    preds = np.asarray(preds)
    targets = np.asarray(targets)
    pred_inds = np.argmax(np.asarray(preds), axis=1)
    return accuracy_score(targets, pred_inds)

def get_avg_f1(preds, labels):
    '''
    Calculate one-vs-all f1 score per class
    Return average of f1 scores over all classes
    preds: [num_samples x num_classes]
    '''
    f1s = []
    label_inds = labels
    class_inds_to_check = list(set(label_inds))

    for class_ind_to_check in class_inds_to_check:
        y_true = []
        y_pred = []
        for pred, lab_ind in zip(preds, label_inds):
            y_pred.append(pred[class_ind_to_check])
            if lab_ind == class_ind_to_check:
                y_true.append(1)
            else:
                y_true.append(0)
        precision, recall, _ = precision_recall_curve(y_true, y_pred)
        f_scores = (2*precision*recall)/(precision+recall)
        f_scores_clean = f_scores[np.logical_not(np.isnan(f_scores))]
        f1s.append(np.amax(f_scores_clean))
    return np.mean(np.asarray(f1s))
    

In [13]:
# evaluate on test set
preds = eval(test_dl, model, device)
targets = [lab_to_ind[lab] for lab in df_test['fact_cwsm_class']]

In [15]:
from scipy.special import softmax

out_dir = './predictions/ResNet/'
dataset = 'dev_in/'
# save targets
np.save(out_dir+dataset+'targets.npy', np.asarray(targets))
# save predictions to file
probs = softmax(preds, axis=1)
np.save(out_dir+dataset+str(seed)+'.npy', np.asarray(probs))

In [16]:
accuracy = metric_accuracy(probs, targets)
f_macro = get_avg_f1(probs, targets)
print(accuracy)
print(f_macro)

/home/miproj/4thyr.oct2019/vr311/venv_tabTrans/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide


0.6671
0.4608502975481431
